# ConveyorGuard - 06. 앙상블 모델

**ML 4종 + DL 2종 앙상블 → 최고 성능 달성**

- **Accelerator: GPU T4 x2**
- 입력: `conveyorguard-preprocess`, `conveyorguard-baseline`, `conveyorguard-ml`, `conveyorguard-tuned`
- 출력: `ensemble_results.csv`, `ensemble_model.pkl`

---

### Pipeline
> `00_eda` → `01_preprocess` → `02/03`(병렬) → `04_dl_tuning` → `05_llm` → **`06_ensemble`** → `07_final`

| 입력 데이터셋 | 출력 | Accelerator |
|--------------|------|-------------|
| 02/03/04 출력물 | `ensemble_results.csv` | GPU T4 x2 |

### 앙상블 전략
```
ML 4종 (XGBoost, LightGBM, RF, CatBoost)
  + DL 2종 (Baseline CNN, Tuned CNN)
  → Soft Voting / Weighted Voting / Stacking
```

## 1. 환경 설정

In [4]:
!pip install plotly -q

import os, sys, time
import numpy as np
import pandas as pd
import joblib
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go
import plotly.figure_factory as ff

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Device: {device}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        mem = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
        print(f"   GPU {i}: {props.name} ({mem / 1024**3:.1f} GB)")

torch.backends.cudnn.benchmark = True
print(f"\n✅ 라이브러리 로드 완료")
_notebook_start = time.time()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA: 12.6

🖥️ Device: cuda
   GPU 0: Tesla T4 (14.6 GB)
   GPU 1: Tesla T4 (14.6 GB)

✅ 라이브러리 로드 완료


In [5]:
DATA_DIR = Path('/kaggle/input/conveyorguard-preprocess')
if not DATA_DIR.exists(): DATA_DIR = Path('/kaggle/input/preprocess')
BASELINE_DIR = Path('/kaggle/input/conveyorguard-baseline')
if not BASELINE_DIR.exists(): BASELINE_DIR = Path('/kaggle/input/baseline')
ML_DIR = Path('/kaggle/input/conveyorguard-ml')
if not ML_DIR.exists(): ML_DIR = Path('/kaggle/input/ml-baseline')
TUNED_DIR = Path('/kaggle/input/conveyorguard-tuned')
if not TUNED_DIR.exists(): TUNED_DIR = Path('/kaggle/input/dl-tuning')
OUTPUT_DIR = Path('/kaggle/working')

for name, d in [('전처리', DATA_DIR), ('Baseline', BASELINE_DIR), ('ML', ML_DIR), ('Tuned', TUNED_DIR)]:
    print(f"📂 {name}: {d} ({d.exists()})")

📂 전처리: /kaggle/input/conveyorguard-preprocess (True)
📂 Baseline: /kaggle/input/conveyorguard-baseline (True)
📂 ML: /kaggle/input/conveyorguard-ml (True)
📂 Tuned: /kaggle/input/conveyorguard-tuned (True)


## 2. 데이터 로드

In [6]:
train_data = joblib.load(DATA_DIR / 'train_data.joblib')
val_data = joblib.load(DATA_DIR / 'val_data.joblib')
test_data = joblib.load(DATA_DIR / 'test_data.joblib')

y_train, y_val, y_test = train_data['labels'], val_data['labels'], test_data['labels']
print(f"Train: {len(y_train):,} / Val: {len(y_val):,} / Test: {len(y_test):,}")

STATE_LABELS = ['정상', '경미', '중간', '심각']

Train: 7,311 / Val: 1,554 / Test: 1,608


## 3. ML 모델 로드 & 예측

In [7]:
def extract_features(data: dict) -> np.ndarray:
    """
    멀티모달 데이터를 ML용 피처로 변환
    
    Input:
        sensors: (N, 30, 8)   → flatten + 통계
        images: (N, 30, 60, 80) → 통계값
        external: (N, 30, 3)  → flatten + 통계
    
    Output:
        features: (N, num_features)
    """
    N = data['sensors'].shape[0]
    features_list = []
    
    # 1. 센서 피처 (8개 센서 × 30 타임스텝)
    sensors = data['sensors']  # (N, 30, 8)
    
    # 센서별 통계 (mean, std, max, min, last)
    sensor_mean = sensors.mean(axis=1)  # (N, 8)
    sensor_std = sensors.std(axis=1)    # (N, 8)
    sensor_max = sensors.max(axis=1)    # (N, 8)
    sensor_min = sensors.min(axis=1)    # (N, 8)
    sensor_last = sensors[:, -1, :]     # (N, 8) - 마지막 값
    sensor_diff = sensors[:, -1, :] - sensors[:, 0, :]  # (N, 8) - 변화량
    
    features_list.extend([sensor_mean, sensor_std, sensor_max, sensor_min, sensor_last, sensor_diff])
    
    # 2. 이미지 피처 (열화상 → 통계값)
    images = data['images']  # (N, 30, 60, 80)
    
    # 프레임별 통계 후 시계열 통계
    img_frame_mean = images.mean(axis=(2, 3))  # (N, 30)
    img_frame_max = images.max(axis=(2, 3))    # (N, 30)
    img_frame_std = images.std(axis=(2, 3))    # (N, 30)
    
    # 시계열 통계
    img_mean = img_frame_mean.mean(axis=1, keepdims=True)  # (N, 1)
    img_std = img_frame_mean.std(axis=1, keepdims=True)    # (N, 1)
    img_max = img_frame_max.max(axis=1, keepdims=True)     # (N, 1)
    img_max_mean = img_frame_max.mean(axis=1, keepdims=True)  # (N, 1)
    img_last_mean = img_frame_mean[:, -1:]   # (N, 1)
    img_last_max = img_frame_max[:, -1:]     # (N, 1)
    img_trend = img_frame_mean[:, -1:] - img_frame_mean[:, 0:1]  # (N, 1) - 온도 변화 추세
    
    features_list.extend([img_mean, img_std, img_max, img_max_mean, img_last_mean, img_last_max, img_trend])
    
    # 3. 외부환경 피처
    external = data['external']  # (N, 30, 3)
    
    ext_mean = external.mean(axis=1)  # (N, 3)
    ext_std = external.std(axis=1)    # (N, 3)
    ext_last = external[:, -1, :]     # (N, 3)
    
    features_list.extend([ext_mean, ext_std, ext_last])
    
    # 모든 피처 결합
    features = np.concatenate(features_list, axis=1)
    
    return features

print("✅ 피처 추출 함수 정의 완료")

✅ 피처 추출 함수 정의 완료


In [8]:
print('🔧 피처 추출...')
X_train = extract_features(train_data)
X_val = extract_features(val_data)
X_test = extract_features(test_data)

scaler = joblib.load(ML_DIR / 'ml_scaler.pkl')
X_train_s, X_val_s, X_test_s = scaler.transform(X_train), scaler.transform(X_val), scaler.transform(X_test)

ml_names = ['xgboost', 'lightgbm', 'randomforest', 'catboost']
ml_models, ml_probas_test, ml_probas_val, ml_probas_train = {}, {}, {}, {}

for name in ml_names:
    fpath = ML_DIR / f'{name}_model.pkl'
    if fpath.exists():
        model = joblib.load(fpath)
        ml_models[name] = model
        ml_probas_test[name] = model.predict_proba(X_test_s)
        ml_probas_val[name] = model.predict_proba(X_val_s)
        ml_probas_train[name] = model.predict_proba(X_train_s)
        acc = accuracy_score(y_test, model.predict(X_test_s)) * 100
        print(f'   ✅ {name}: {acc:.2f}%')

print(f'\n✅ ML {len(ml_models)}종 로드')

🔧 피처 추출...
   ✅ xgboost: 96.70%
   ✅ lightgbm: 96.89%
   ✅ randomforest: 95.58%
   ✅ catboost: 96.46%

✅ ML 4종 로드


## 4. DL 모델 로드 & 예측

In [12]:
# ============ 02 Baseline용 (Dropout2d 없음, norm 1개) ============
class SensorEncoderBaseline(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(8, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(64, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.GELU()
        )
    def forward(self, x):
        return self.mlp(x)

class ImageEncoderBaseline(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.GELU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.GELU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.GELU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.proj = nn.Linear(128, embed_dim)
    def forward(self, x):
        B, T, H, W = x.shape
        x = x.view(B * T, 1, H, W)
        x = self.cnn(x)
        x = x.view(B * T, -1)
        x = self.proj(x)
        return x.view(B, T, -1)

class TemporalEncoderBaseline(nn.Module):
    def __init__(self, embed_dim=128, n_heads=4, n_layers=2):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.randn(1, 30, embed_dim) * 0.02)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=n_heads, dim_feedforward=embed_dim*4,
            dropout=0.1, activation='gelu', batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
    def forward(self, x):
        x = x + self.pos_embed[:, :x.size(1), :]
        return self.transformer(x)

class CrossAttentionFusionBaseline(nn.Module):
    def __init__(self, embed_dim=128, n_heads=4):
        super().__init__()
        self.cross_attn = nn.MultiheadAttention(embed_dim, n_heads, batch_first=True)
        self.norm = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(embed_dim * 4, embed_dim)
        )
    def forward(self, sensor_feat, image_feat):
        attn_out, _ = self.cross_attn(sensor_feat, image_feat, image_feat)
        x = self.norm(sensor_feat + attn_out)
        x = x + self.ffn(x)
        return x

class ConveyorGuardModelBaseline(nn.Module):
    def __init__(self, embed_dim=128, num_classes=4):
        super().__init__()
        self.sensor_encoder = SensorEncoderBaseline(embed_dim)
        self.image_encoder = ImageEncoderBaseline(embed_dim)
        self.external_encoder = nn.Sequential(nn.Linear(3, embed_dim), nn.LayerNorm(embed_dim), nn.GELU())
        self.sensor_temporal = TemporalEncoderBaseline(embed_dim)
        self.image_temporal = TemporalEncoderBaseline(embed_dim)
        self.fusion = CrossAttentionFusionBaseline(embed_dim)
        self.film_gamma = nn.Linear(embed_dim, embed_dim)
        self.film_beta = nn.Linear(embed_dim, embed_dim)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim), nn.LayerNorm(embed_dim), nn.GELU(),
            nn.Dropout(0.1), nn.Linear(embed_dim, num_classes)
        )
    def forward(self, sensors, images, externals=None):
        sensor_feat = self.sensor_encoder(sensors)
        image_feat = self.image_encoder(images)
        sensor_feat = self.sensor_temporal(sensor_feat)
        image_feat = self.image_temporal(image_feat)
        fused = self.fusion(sensor_feat, image_feat)
        pooled = fused.mean(dim=1)
        if externals is not None:
            ext_feat = self.external_encoder(externals)
            ext_pooled = ext_feat.mean(dim=1)
            gamma = self.film_gamma(ext_pooled)
            beta = self.film_beta(ext_pooled)
            pooled = gamma * pooled + beta
        return self.classifier(pooled)

# ============ 04 Tuned용 (기존 그대로) ============
class SensorEncoder(nn.Module):
    def __init__(self, input_dim=8, embed_dim=128, dropout=0.1):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 64), nn.LayerNorm(64), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(64, embed_dim), nn.LayerNorm(embed_dim), nn.GELU()
        )
    def forward(self, x):
        return self.mlp(x)

class ImageEncoder(nn.Module):
    def __init__(self, embed_dim=128, dropout=0.1):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.GELU(), nn.MaxPool2d(2), nn.Dropout2d(dropout),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.GELU(), nn.MaxPool2d(2), nn.Dropout2d(dropout),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.GELU(), nn.AdaptiveAvgPool2d(1)
        )
        self.proj = nn.Linear(128, embed_dim)
    def forward(self, x):
        B, T, H, W = x.shape
        x = x.view(B * T, 1, H, W)
        x = self.cnn(x)
        x = x.view(B * T, -1)
        x = self.proj(x)
        return x.view(B, T, -1)

class TemporalEncoder(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, seq_len=30):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.randn(1, seq_len, embed_dim) * 0.02)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim*4,
            dropout=dropout, activation='gelu', batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
    def forward(self, x):
        x = x + self.pos_embed[:, :x.size(1), :]
        return self.transformer(x)

class CrossAttentionFusion(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, dropout=0.1):
        super().__init__()
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim*4), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(embed_dim*4, embed_dim), nn.Dropout(dropout)
        )
    def forward(self, sensor_feat, image_feat):
        attn_out, _ = self.cross_attn(sensor_feat, image_feat, image_feat)
        x = self.norm1(sensor_feat + attn_out)
        x = self.norm2(x + self.ffn(x))
        return x

class ConveyorGuardModel(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, num_classes=4, sensor_seq_len=30, image_seq_len=10):
        super().__init__()
        self.sensor_encoder = SensorEncoder(8, embed_dim, dropout)
        self.image_encoder = ImageEncoder(embed_dim, dropout)
        self.external_encoder = nn.Sequential(nn.Linear(3, embed_dim), nn.LayerNorm(embed_dim), nn.GELU())
        self.sensor_temporal = TemporalEncoder(embed_dim, num_heads, num_layers, dropout, sensor_seq_len)
        self.image_temporal = TemporalEncoder(embed_dim, num_heads, num_layers, dropout, image_seq_len)
        self.fusion = CrossAttentionFusion(embed_dim, num_heads, dropout)
        self.film_gamma = nn.Linear(embed_dim, embed_dim)
        self.film_beta = nn.Linear(embed_dim, embed_dim)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim), nn.LayerNorm(embed_dim), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(embed_dim, num_classes)
        )
    def forward(self, sensors, images, externals=None):
        sensor_feat = self.sensor_encoder(sensors)
        image_feat = self.image_encoder(images)
        sensor_feat = self.sensor_temporal(sensor_feat)
        image_feat = self.image_temporal(image_feat)
        fused = self.fusion(sensor_feat, image_feat)
        pooled = fused.mean(dim=1)
        if externals is not None:
            ext_feat = self.external_encoder(externals)
            ext_pooled = ext_feat.mean(dim=1)
            pooled = self.film_gamma(ext_pooled) * pooled + self.film_beta(ext_pooled)
        return self.classifier(pooled)

print("✅ 모델 클래스 정의 완료 (Baseline + Tuned)")

✅ 모델 클래스 정의 완료 (Baseline + Tuned)


In [13]:
class ConveyorDataset(Dataset):
    def __init__(self, data: dict):
        self.sensors = torch.FloatTensor(data['sensors'])
        self.images = torch.FloatTensor(data['images'])
        self.externals = torch.FloatTensor(data['external'])  # 키 이름 통일: 'external'
        self.labels = torch.LongTensor(data['labels'])
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'sensors': self.sensors[idx],
            'images': self.images[idx],
            'externals': self.externals[idx],
            'label': self.labels[idx]
        }

# Dataset 생성
train_dataset = ConveyorDataset(train_data)
val_dataset = ConveyorDataset(val_data)

print(f"✅ Dataset 생성 완료")
print(f"   Train: {len(train_dataset)}개")
print(f"   Val: {len(val_dataset)}개")

class ConveyorDatasetOptimized(Dataset):
    """
    최적화된 Dataset
    - 이미지: 30프레임 → 10프레임 서브샘플링 (3배 빨라짐)
    """
    def __init__(self, data: dict, image_frames: int = 10):
        self.sensors = torch.FloatTensor(data['sensors'])
        
        # ⚡ 이미지 서브샘플링 (30 → 10 프레임)
        images = data['images']
        indices = np.linspace(0, images.shape[1]-1, image_frames, dtype=int)
        self.images = torch.FloatTensor(images[:, indices, :, :])
        
        self.externals = torch.FloatTensor(data['external'])
        self.labels = torch.LongTensor(data['labels'])
        
        self.image_frames = image_frames
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'sensors': self.sensors[idx],
            'images': self.images[idx],
            'externals': self.externals[idx],
            'label': self.labels[idx]
        }

# Dataset 생성
IMAGE_FRAMES = 10  # 30 → 10 (최적화)

train_dataset = ConveyorDatasetOptimized(train_data, image_frames=IMAGE_FRAMES)
val_dataset = ConveyorDatasetOptimized(val_data, image_frames=IMAGE_FRAMES)
test_dataset = ConveyorDatasetOptimized(test_data, image_frames=IMAGE_FRAMES)

print(f"✅ Dataset 생성 완료 (이미지 {IMAGE_FRAMES}프레임)")
print(f"   Train: {len(train_dataset):,}개")
print(f"   Val: {len(val_dataset):,}개")
print(f"   Test: {len(test_dataset):,}개")
print(f"   이미지 shape: {train_dataset.images.shape}")

✅ Dataset 생성 완료
   Train: 7311개
   Val: 1554개
✅ Dataset 생성 완료 (이미지 10프레임)
   Train: 7,311개
   Val: 1,554개
   Test: 1,608개
   이미지 shape: torch.Size([7311, 10, 60, 80])


In [14]:
BATCH_SIZE = 64

baseline_ckpt = torch.load(BASELINE_DIR / 'baseline_cnn_model.pt', map_location=device)
baseline_model = ConveyorGuardModelBaseline(embed_dim=128, num_classes=4).to(device)
baseline_model.load_state_dict(baseline_ckpt['model_state_dict'])
baseline_model.eval()
print(f'✅ Baseline (Best Acc: {baseline_ckpt.get("best_acc", "N/A")}) [image_seq_len=30]')

tuned_ckpt = torch.load(TUNED_DIR / 'tuned_model.pt', map_location=device)
tc = tuned_ckpt['model_config']
tuned_model = ConveyorGuardModel(
    embed_dim=tc['embed_dim'], num_heads=tc.get('num_heads',4),
    num_layers=tc.get('num_layers',1), dropout=tc.get('dropout',0.1),
    num_classes=4, image_seq_len=tc.get('image_seq_len',10)
).to(device)
tuned_model.load_state_dict(tuned_ckpt['model_state_dict'])
tuned_model.eval()
print(f'✅ Tuned (Val Acc: {tuned_ckpt.get("best_val_acc", "N/A")}) [image_seq_len={tc.get("image_seq_len",10)}]')

def get_dl_probas(model, data, image_frames=None):
    ds = ConveyorDatasetOptimized(data, image_frames) if image_frames else ConveyorDataset(data)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    all_p = []
    with torch.no_grad():
        for batch in loader:
            s = batch['sensors'].to(device, non_blocking=True)
            im = batch['images'].to(device, non_blocking=True)
            ex = batch['externals'].to(device, non_blocking=True)
            with autocast():
                logits = model(s, im, ex)
            all_p.append(F.softmax(logits, dim=1).cpu().numpy())
    return np.concatenate(all_p)

img_f = tc.get('image_seq_len', 10)
dl_probas, dl_probas_val, dl_probas_train = {}, {}, {}
for nm, mdl, ifr in [('baseline', baseline_model, None), ('tuned', tuned_model, img_f)]:
    dl_probas[nm] = get_dl_probas(mdl, test_data, ifr)
    dl_probas_val[nm] = get_dl_probas(mdl, val_data, ifr)
    dl_probas_train[nm] = get_dl_probas(mdl, train_data, ifr)
    print(f'   ✅ {nm}: {accuracy_score(y_test, dl_probas[nm].argmax(1))*100:.2f}%')

✅ Baseline (Best Acc: 93.24324324324324) [image_seq_len=30]
✅ Tuned (Val Acc: 90.34749034749035) [image_seq_len=10]
   ✅ baseline: 92.72%
   ✅ tuned: 87.75%


## 5. 개별 모델 성능 확인

In [15]:
all_probas_test = {**ml_probas_test, **dl_probas}
all_probas_val = {**ml_probas_val, **dl_probas_val}

individual_results = []
for name, proba in all_probas_test.items():
    preds = proba.argmax(1)
    val_preds = all_probas_val[name].argmax(1)
    individual_results.append({
        'Model': name, 'Val_Acc': accuracy_score(y_val, val_preds)*100,
        'Test_Acc': accuracy_score(y_test, preds)*100,
        'Test_F1': f1_score(y_test, preds, average='weighted')*100,
    })

ind_df = pd.DataFrame(individual_results).sort_values('Test_Acc', ascending=False)
print('📊 개별 모델 성능:')
print(ind_df.to_string(index=False))

📊 개별 모델 성능:
       Model   Val_Acc  Test_Acc   Test_F1
    lightgbm 96.975547 96.890547 96.890147
     xgboost 97.039897 96.703980 96.704233
    catboost 96.589447 96.455224 96.456080
randomforest 96.525097 95.584577 95.631784
    baseline 93.243243 92.723881 92.799636
       tuned 88.095238 87.748756 88.129900


## 6. 앙상블 전략

### 6.1 Soft Voting (균등 가중)

In [16]:
all_test_probas = list(all_probas_test.values())
soft_avg = np.mean(all_test_probas, axis=0)
soft_preds = soft_avg.argmax(1)
soft_acc = accuracy_score(y_test, soft_preds)*100
soft_f1 = f1_score(y_test, soft_preds, average='weighted')*100
print(f'🎯 Soft Voting: Acc={soft_acc:.2f}%, F1={soft_f1:.2f}%')
print(classification_report(y_test, soft_preds, target_names=STATE_LABELS))

🎯 Soft Voting: Acc=96.64%, F1=96.64%
              precision    recall  f1-score   support

          정상       0.99      0.99      0.99       788
          경미       0.93      0.94      0.93       371
          중간       0.96      0.94      0.95       361
          심각       0.97      0.98      0.97        88

    accuracy                           0.97      1608
   macro avg       0.96      0.96      0.96      1608
weighted avg       0.97      0.97      0.97      1608



### 6.2 Weighted Voting (Val Acc 기반)

In [17]:
weights = np.array([accuracy_score(y_val, all_probas_val[n].argmax(1)) for n in all_probas_test])
weights = weights / weights.sum()
for n, w in zip(all_probas_test, weights): print(f'   {n}: {w:.4f}')

weighted_avg = np.average(all_test_probas, axis=0, weights=weights)
weighted_preds = weighted_avg.argmax(1)
weighted_acc = accuracy_score(y_test, weighted_preds)*100
weighted_f1 = f1_score(y_test, weighted_preds, average='weighted')*100
print(f'\n🎯 Weighted: Acc={weighted_acc:.2f}%, F1={weighted_f1:.2f}%')
print(classification_report(y_test, weighted_preds, target_names=STATE_LABELS))

   xgboost: 0.1707
   lightgbm: 0.1706
   randomforest: 0.1698
   catboost: 0.1699
   baseline: 0.1640
   tuned: 0.1550

🎯 Weighted: Acc=96.70%, F1=96.71%
              precision    recall  f1-score   support

          정상       0.99      0.99      0.99       788
          경미       0.93      0.94      0.94       371
          중간       0.96      0.94      0.95       361
          심각       0.97      0.98      0.97        88

    accuracy                           0.97      1608
   macro avg       0.96      0.96      0.96      1608
weighted avg       0.97      0.97      0.97      1608



### 6.3 Stacking (Meta-Learner)

In [18]:
all_train_probas = {**ml_probas_train, **dl_probas_train}
meta_train = np.hstack([all_train_probas[n] for n in all_probas_test])
meta_val = np.hstack([all_probas_val[n] for n in all_probas_test])
meta_test = np.hstack([all_probas_test[n] for n in all_probas_test])
print(f'meta-features: {meta_train.shape[1]} ({len(all_probas_test)}모델×4클래스)')

meta_learner = LogisticRegression(max_iter=1000, random_state=42)
meta_learner.fit(meta_train, y_train)

stack_preds = meta_learner.predict(meta_test)
stack_acc = accuracy_score(y_test, stack_preds)*100
stack_f1 = f1_score(y_test, stack_preds, average='weighted')*100
meta_val_acc = accuracy_score(y_val, meta_learner.predict(meta_val))*100
print(f'\n🎯 Stacking: Val={meta_val_acc:.2f}%, Test={stack_acc:.2f}%, F1={stack_f1:.2f}%')
print(classification_report(y_test, stack_preds, target_names=STATE_LABELS))

meta-features: 24 (6모델×4클래스)

🎯 Stacking: Val=96.91%, Test=96.89%, F1=96.89%
              precision    recall  f1-score   support

          정상       0.99      0.99      0.99       788
          경미       0.93      0.94      0.94       371
          중간       0.96      0.94      0.95       361
          심각       0.98      0.98      0.98        88

    accuracy                           0.97      1608
   macro avg       0.96      0.96      0.96      1608
weighted avg       0.97      0.97      0.97      1608



## 7. 결과 비교

In [19]:
ensemble_results = individual_results.copy()
ensemble_results.append({'Model':'Soft Voting','Val_Acc':None,'Test_Acc':soft_acc,'Test_F1':soft_f1})
ensemble_results.append({'Model':'Weighted Voting','Val_Acc':None,'Test_Acc':weighted_acc,'Test_F1':weighted_f1})
ensemble_results.append({'Model':'Stacking','Val_Acc':meta_val_acc,'Test_Acc':stack_acc,'Test_F1':stack_f1})
results_df = pd.DataFrame(ensemble_results).sort_values('Test_Acc', ascending=False)
print(results_df.to_string(index=False))

colors = ['#EF4444' if any(k in m for k in ['Voting','Stacking']) else '#3B82F6' for m in results_df['Model']]
fig = go.Figure(go.Bar(x=results_df['Model'], y=results_df['Test_Acc'],
    marker_color=colors, text=results_df['Test_Acc'].round(2), textposition='outside'))
fig.update_layout(title='모델별 Test Accuracy (개별 vs 앙상블)',
    xaxis_title='Model', yaxis_title='Acc (%)', height=500, width=900,
    yaxis=dict(range=[max(0,results_df['Test_Acc'].min()-5), 100]))
fig.show(renderer='iframe')

          Model   Val_Acc  Test_Acc   Test_F1
       lightgbm 96.975547 96.890547 96.890147
       Stacking 96.911197 96.890547 96.890923
        xgboost 97.039897 96.703980 96.704233
Weighted Voting       NaN 96.703980 96.707115
    Soft Voting       NaN 96.641791 96.644509
       catboost 96.589447 96.455224 96.456080
   randomforest 96.525097 95.584577 95.631784
       baseline 93.243243 92.723881 92.799636
          tuned 88.095238 87.748756 88.129900


In [20]:
best_row = results_df.iloc[0]
bm = best_row['Model']
if 'Stacking' in bm: bp = stack_preds
elif 'Weighted' in bm: bp = weighted_preds
elif 'Soft' in bm: bp = soft_preds
else: bp = all_probas_test[bm].argmax(1)

cm = confusion_matrix(y_test, bp)
fig = ff.create_annotated_heatmap(z=cm, x=STATE_LABELS, y=STATE_LABELS,
    colorscale='Blues', showscale=True)
fig.update_layout(title=f'Confusion Matrix — {bm} ({best_row["Test_Acc"]:.2f}%)',
    xaxis_title='Predicted', yaxis_title='Actual', width=600, height=500)
fig.update_yaxes(autorange='reversed')
fig.show(renderer='iframe')

## 8. 모델 저장

In [21]:
results_df.to_csv(OUTPUT_DIR / 'ensemble_results.csv', index=False, encoding='utf-8-sig')
joblib.dump(meta_learner, OUTPUT_DIR / 'ensemble_model.pkl', compress=3)
print(f'✅ 저장 완료')
print(f'🏆 Best: {best_row["Model"]} → {best_row["Test_Acc"]:.2f}%')

✅ 저장 완료
🏆 Best: lightgbm → 96.89%


In [22]:
_total_time = time.time() - _notebook_start
print(f"\n⏱️ 총 실행 시간: {_total_time/60:.1f}분 ({_total_time:.0f}초)")
print('\n✅ 06_ensemble 완료! 다음: 07_final_comparison.ipynb')


⏱️ 총 실행 시간: 14.2분 (853초)

✅ 06_ensemble 완료! 다음: 07_final_comparison.ipynb


## Kaggle 데이터셋 업로드 (API)

In [23]:
import os, json

os.environ['KAGGLE_USERNAME'] = 'kukass'
os.environ['KAGGLE_KEY'] = 'KGAT_c973fff8eb3e1ccb19f3e9d683eb17dc'

UPLOAD_DIR = '/kaggle/working/dataset_upload'
os.makedirs(UPLOAD_DIR, exist_ok=True)

output_files = ['ensemble_results.csv', 'ensemble_model.pkl']
for f in output_files:
    src = f'/kaggle/working/{f}'
    dst = f'{UPLOAD_DIR}/{f}'
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)

meta = {
    "title": "conveyorguard-ensemble",
    "id": "kukass/conveyorguard-ensemble",
    "licenses": [{"name": "CC0-1.0"}]
}
with open(f'{UPLOAD_DIR}/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

!kaggle datasets create -p {UPLOAD_DIR} --dir-mode zip

print("\n✅ conveyorguard-ensemble 데이터셋 업로드 완료!")
print("   → 07_final_comparison에서 사용 가능")

Starting upload for file ensemble_results.csv
100%|██████████████████████████████████████████| 574/574 [00:00<00:00, 1.50kB/s]
Upload successful: ensemble_results.csv (574B)
Starting upload for file ensemble_model.pkl
100%|██████████████████████████████████████| 1.38k/1.38k [00:00<00:00, 3.82kB/s]
Upload successful: ensemble_model.pkl (1KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/kukass/conveyorguard-ensemble

✅ conveyorguard-ensemble 데이터셋 업로드 완료!
   → 07_final_comparison에서 사용 가능


---

## 📋 결과 요약

| 전략 | 설명 |
|----|----|
| **Soft Voting** | 균등 가중 평균 |
| **Weighted Voting** | Val Acc 비례 가중 |
| **Stacking** | Meta-learner (LR) |

### 다음 단계
- `07_final_comparison.ipynb`: 전체 모델 비교 + 최종 결론